# Databases

![Data Science Workflow](img/ds-workflow.png)

## Acquire Data
### Common Data Sources
- The Internet - Web Scraping
- **Databases**
- CSV
- Excel
- Parquet

### Database
- **Relational databases** models data in rows and columns in a series of tables.
- Like a collection of **DataFrames/Excel sheets**.
- **SQL** (often pronounced *sequel*): Structured Query Language
- Non-relational databases [NoSQL](https://en.wikipedia.org/wiki/NoSQL)

### SQLite database
- [SQLite database](https://www.sqlite.org/index.html) software library that provides a relational database management system
- Lightweight to setup, administrate, and requires low resources.

### Dataset (in a SQLite database)
- Dallas Police Officer-Involved Shootings
- Has three tables: **incidents, officers, subjects**

#### Other SQLite datasets
- [SQLite Data Starter Packs](http://2016.padjo.org/tutorials/sqlite-data-starterpacks/)
- [Iris Species (Kaggle)](https://www.kaggle.com/uciml/iris)
- [European Soccer Database (Kaggle)](https://www.kaggle.com/hugomathien/soccer)

### Database connector
- The [sqlite3](https://docs.python.org/3/library/sqlite3.html) is an interface for SQLite databases.
- No installation needed.

#### Other database connectors for Python
* [MySQL Connector](https://dev.mysql.com/doc/connector-python/en/) for MySQL
* [Psycopg](http://initd.org/psycopg/docs/install.html) for PostgreSQL
* [pymssql](http://pymssql.org/en/stable/) for Microsoft MS SQL

To import and connect to a SQLite database.
```Python
import sqlite3
conn = sqlite3.connect('files/dallas-ois.sqlite')
```

In [1]:
import sqlite3

In [4]:
conn =sqlite3.connect('files/dallas-ois.sqlite')

The interface to the datases is through the `cursor`.

```Python
cursor = conn.cursor()
```

The `execute` method allows to run `SQL` queries on our database.

To get a list of all the tables the following query can be applied.

```SQL
SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
```

The result is fetched with the `fetchall()` method.

In [5]:
cursor = conn.cursor()

In [6]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")

In [7]:
cursor.fetchall()

[('incidents',), ('officers',), ('subjects',)]

To get a description of a table (SQLite specific)

```SQL
PRAGMA table_info(officers)
```

In [8]:
cursor.execute("PRAGMA table_info(officers)").fetchall()

[(0, 'case_number', 'TEXT', 0, None, 0),
 (1, 'race', 'TEXT', 0, None, 0),
 (2, 'gender', 'TEXT', 0, None, 0),
 (3, 'last_name', 'TEXT', 0, None, 0),
 (4, 'first_name', 'TEXT', 0, None, 0),
 (5, 'full_name', 'TEXT', 0, None, 0)]

In [9]:
cursor.execute("PRAGMA table_info(incidents)").fetchall()

[(0, 'case_number', 'TEXT', 0, None, 0),
 (1, 'date', 'DATE', 0, None, 0),
 (2, 'location', 'TEXT', 0, None, 0),
 (3, 'subject_statuses', 'TEXT', 0, None, 0),
 (4, 'subject_weapon', 'TEXT', 0, None, 0),
 (5, 'subjects', 'TEXT', 0, None, 0),
 (6, 'subject_count', 'INTEGER', 0, None, 0),
 (7, 'officers', 'TEXT', 0, None, 0),
 (8, 'officer_count', 'INTEGER', 0, None, 0),
 (9, 'grand_jury_disposition', 'TEXT', 0, None, 0),
 (10, 'attorney_general_forms_url', 'TEXT', 0, None, 0),
 (11, 'summary_url', 'TEXT', 0, None, 0),
 (12, 'summary_text', 'TEXT', 0, None, 0),
 (13, 'latitude', 'FLOAT', 0, None, 0),
 (14, 'longitude', 'FLOAT', 0, None, 0)]

### SQL syntax
Get all data from table
```SQL
SELECT * FROM table_name
```

Limit the response
```SQL
SELECT * FROM table_name LIMIT 100
```

Filter data
```SQL
SELECT * FROM table_name WHERE column_name > 1
```

### Import data into DataFrame
Example of how to read data into a DataFrame from a SQL statement
```Python
import pandas as pd

officers = pd.read_sql('SELECT * FROM officers', conn)
```

In [10]:
import pandas as pd

In [11]:
officers = pd.read_sql('SELECT * FROM officers', conn)

In [12]:
officers.head(2)

case_number race gender  last_name first_name         full_name
0      44523A    L      M     Patino    Michael   Patino, Michael
1      44523A    W      M  Fillingim      Brian  Fillingim, Brian

### SQL join syntax
`(INNER) JOIN`: returns records that have matching values in both tables
```SQL
SELECT * FROM table_1 JOIN table_2 ON table_1.column_name_1=table_2.column_name_2
```

`LEFT JOIN`: returns all records from the left table, and the matched records from the right table
```SQL
SELECT * FROM table_1 LEFT JOIN table_2 ON table_1.column_name_1=table_2.column_name_2
```

In [14]:
officers = pd.read_sql("SELECT * FROM officers JOIN incidents ON officers.case_number=incidents.case_number", conn)

In [15]:
officers.head()

case_number race gender  last_name first_name         full_name case_number  \
0      44523A    L      M     Patino    Michael   Patino, Michael      44523A   
1      44523A    W      M  Fillingim      Brian  Fillingim, Brian      44523A   
2     121982X    L      M    Padilla    Gilbert  Padilla, Gilbert     121982X   
3     605484T    W      M     Poston      Jerry     Poston, Jerry     605484T   
4     384832T    B      M      Mondy    Michael    Mondy, Michael     384832T   

         date                  location subject_statuses  ...  \
0  2013-02-23     3000 Chihuahua Street          Injured  ...   
1  2013-02-23     3000 Chihuahua Street          Injured  ...   
2  2010-05-03  1300 N. Munger Boulevard          Injured  ...   
3  2007-08-12   200 S. Stemmons Freeway            Other  ...   
4  2007-05-26           7900 S. Loop 12   Shoot and Miss  ...   

                                            subjects subject_count  \
0                                   Curry, James L/M             1   
1                                   Curry, James L/M             1   
2                                Chavez, Gabriel L/M             1   
3                                  Salinas, Nick L/M             1   
4  Smith, James B/M; Dews, Antonio B/M; Spearman,...             3   

                                    officers officer_count  \
0  Patino, Michael L/M; Fillingim, Brian W/M             2   
1  Patino, Michael L/M; Fillingim, Brian W/M             2   
2                       Padilla, Gilbert L/M             1   
3                          Poston, Jerry W/M             1   
4                         Mondy, Michael B/M             1   

   grand_jury_disposition attorney_general_forms_url  \
0                 No Bill                       None   
1                 No Bill                       None   
2                 No Bill                       None   
3             See Summary                       None   
4                    None                       None   

                                         summary_url  \
0  http://dallaspolice.net/reports/OIS/narrative/...   
1  http://dallaspolice.net/reports/OIS/narrative/...   
2  http://dallaspolice.net/reports/OIS/narrative/...   
3  http://dallaspolice.net/reports/OIS/narrative/...   
4  http://dallaspolice.net/reports/OIS/narrative/...   

                                        summary_text latitude  longitude  
0  On Saturday, February 23, 2013, at approximate...      NaN        NaN  
1  On Saturday, February 23, 2013, at approximate...      NaN        NaN  
2  On Monday, May 3, 2010, at approximately 8:06 ...      NaN        NaN  
3  On Sunday, August 12, 2007, at approximately 2...      NaN        NaN  
4  On Saturday, May 26, 2007, at approximately 1:...      NaN        NaN  

[5 rows x 21 columns]

In [16]:
incidents = pd.read_sql('SELECT * FROM incidents',conn)

In [22]:
incidents.dtypes

case_number                    object
date                           object
location                       object
subject_statuses               object
subject_weapon                 object
subjects                       object
subject_count                   int64
officers                       object
officer_count                   int64
grand_jury_disposition         object
attorney_general_forms_url     object
summary_url                    object
summary_text                   object
latitude                      float64
longitude                     float64
dtype: object

In [18]:
subjects = pd.read_sql('SELECT * FROM subjects',conn)

In [23]:
subjects.dtypes

case_number    object
race           object
gender         object
last_name      object
first_name     object
full_name      object
dtype: object